# 🎓 Particle-Particle interaction


In this tutorial, we show an example of the 'particle-particle-interaction' functionality in Parcels. Note that this functionality is still fairly rudimentary for now. Importantly, as particle-interaction is a many-body problem it scales as $N^2$ and can thus become very slow for large ParticleSets. There is currently **no advanced optimisation** (such as using techniques from [Smoothed Particle Hydrodynamics](https://en.wikipedia.org/wiki/Smoothed-particle_hydrodynamics)) build-in to Parcels. 

## Pulling particles

Below is an example of what can be done with particle-particle interaction. We create a square grid of $N\times N$ particles, which are all subject to Brownian Motion (via the built-in `DiffusionUniformKh` Kernel). Furthermore, two of the particles also 'attract' other particles that are within the interaction distance: these attracted particles move with a constant velocity to the attracting particles.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from matplotlib.animation import FuncAnimation

import parcels

# for interactive display of animations
plt.rcParams["animation.html"] = "jshtml"

In [ ]:
def Pull(particles, fieldset):
    """Kernel that "pulls" all neighbour particles
    toward the attracting particle with a constant velocity"""
    interaction_distance = 0.5
    velocity = -0.04  # predefined attracting velocity

    # Boolean mask and coordinates of attractors
    attractor_mask = particles.attractor.astype(bool)
    lon_a = particles.lon[attractor_mask]
    lat_a = particles.lat[attractor_mask]

    # Pairwise differences and distances (n_attractors × n_particles)
    dx = particles.lon - lon_a[:, None]
    dy = particles.lat - lat_a[:, None]
    distances = np.sqrt(dx**2 + dy**2)

    # Mask dx, dy by interaction range
    within = distances < interaction_distance
    dx = np.where(within, dx, 0.0)
    dy = np.where(within, dy, 0.0)

    with np.errstate(divide="ignore", invalid="ignore"):
        inv_dist = np.where(distances > 0, 1.0 / distances, 0.0)
        dx_norm = dx * inv_dist
        dy_norm = dy * inv_dist

    particles.dlon += np.sum(dx_norm, axis=0) * velocity * particles.dt
    particles.dlat += np.sum(dy_norm, axis=0) * velocity * particles.dt

In [ ]:
def DiffusionFieldSet():
    """Define a fieldset with only diffusion"""
    fieldset = parcels.FieldSet([])
    fieldset.add_constant_field("Kh_zonal", 0.0005, mesh="flat")
    fieldset.add_constant_field("Kh_meridional", 0.0005, mesh="flat")
    return fieldset

In [ ]:
npart = 11
X, Y = np.meshgrid(np.linspace(-1, 1, npart), np.linspace(-1, 1, npart))

# Create custom particle class with extra variable that indicates
# whether the interaction kernel should be executed on this particle.
InteractingParticle = parcels.Particle.add_variable(
    parcels.Variable("attractor", dtype=np.bool_, to_write="once"),
)

attractor = [
    True if i in [int(npart * npart / 2 - 3), int(npart * npart / 2 + 3)] else False
    for i in range(npart * npart)
]

pset = parcels.ParticleSet(
    fieldset=DiffusionFieldSet(),
    pclass=InteractingParticle,
    lon=X,
    lat=Y,
    attractor=attractor,
)

output_file = parcels.ParticleFile(
    store="InteractingParticles.zarr",
    outputdt=np.timedelta64(1, "s"),
)

kernels = [
    parcels.kernels.DiffusionUniformKh,
    Pull,  # Add the Pull kernel defined above
]

pset.execute(
    pyfunc=kernels,
    runtime=np.timedelta64(60, "s"),
    dt=np.timedelta64(1, "s"),
    output_file=output_file,
    verbose_progress=False,
)

In [ ]:
data_xarray = xr.open_zarr("InteractingParticles.zarr")
data_attr = data_xarray.where(data_xarray["attractor"].compute() == 1, drop=True)
data_other = data_xarray.where(data_xarray["attractor"].compute() == 0, drop=True)

timerange = np.arange(
    np.nanmin(data_xarray["time"].values),
    np.nanmax(data_xarray["time"].values),
    np.timedelta64(1, "s"),
)

fig = plt.figure(figsize=(4, 4), constrained_layout=True)
ax = fig.add_subplot()

ax.set_ylabel("Meridional distance [m]")
ax.set_xlabel("Zonal distance [m]")
ax.set_xlim(-1.1, 1.1)
ax.set_ylim(-1.1, 1.1)

time_id = np.where(data_other["time"] == timerange[0])
time_id_attr = np.where(data_attr["time"] == timerange[0])

scatter = ax.scatter(
    data_other["lon"].values[time_id],
    data_other["lat"].values[time_id],
    c="b",
    s=5,
    zorder=1,
)
scatter_attr = ax.scatter(
    data_attr["lon"].values[time_id_attr],
    data_attr["lat"].values[time_id_attr],
    c="r",
    s=40,
    zorder=2,
)

circs = []
for lon_a, lat_a in zip(
    data_attr["lon"].values[time_id_attr], data_attr["lat"].values[time_id_attr]
):
    circs.append(
        ax.add_patch(
            plt.Circle(
                (lon_a, lat_a), 0.25, facecolor="None", edgecolor="r", linestyle="--"
            )
        )
    )

t = str(timerange[0].astype("timedelta64[s]"))
title = ax.set_title("Particles at t = " + t + " (Red particles are attractors)")


def animate(i):
    t = str(timerange[i].astype("timedelta64[s]"))
    title.set_text("Particles at t = " + t + "\n (Red particles are attractors)")

    time_id = np.where(data_other["time"] == timerange[i])
    time_id_attr = np.where(data_attr["time"] == timerange[i])
    scatter.set_offsets(
        np.c_[data_other["lon"].values[time_id], data_other["lat"].values[time_id]]
    )
    scatter_attr.set_offsets(
        np.c_[
            data_attr["lon"].values[time_id_attr], data_attr["lat"].values[time_id_attr]
        ]
    )
    for c, lon_a, lat_a in zip(
        circs,
        data_attr["lon"].values[time_id_attr],
        data_attr["lat"].values[time_id_attr],
    ):
        c.center = (lon_a, lat_a)


# Create animation
anim = FuncAnimation(fig, animate, frames=len(timerange), interval=100)
plt.close(fig)
anim

## Merging particles

Another type of interaction is the merging of particles. The merging Kernel also comes with limitations (only mutual-nearest particles can be accurately merged), so this is really just a prototype. Nevertheless, the example below shows the possibilities that merging of particles can provide for more complex simulations.

In [ ]:
def Merge(particles, fieldset):
    """Kernel that "merges" two particles that are within interaction_distance range
    Particles must have a 'mass' Variable, and during merging the mass of the smallest
    particle is transferred to the largest particle. The smallest particle is then deleted.
    """
    interaction_distance = 0.05

    N = len(particles.lon)

    # calculate pairwise distances (n_particles × n_particles)
    dx = particles.lon[None, :] - particles.lon[:, None]
    dy = particles.lat[None, :] - particles.lat[:, None]
    distances = np.sqrt(dx**2 + dy**2)

    # mask distances by interaction range
    within = (distances > 0) & (distances < interaction_distance)
    dist_masked = np.where(within, distances, np.inf)

    # nearest neighbour for each particle (index) and corresponding distance
    nn = np.argmin(dist_masked, axis=1)
    nn_dist = dist_masked[np.arange(N), nn]
    has_nn = np.isfinite(nn_dist)

    # mutual nearest: nn[nn[i]] == i
    i_idx = np.arange(N)
    mutual = has_nn & (nn[nn] == i_idx)

    # define pairs i and their nearest neighbour j
    pair_mask = mutual & (i_idx < nn)
    pair_i = i_idx[pair_mask]
    pair_j = nn[pair_mask]

    if pair_i.size == 0:
        return

    # compute which of the two (i, j) has the largest mass
    mass_i = particles.mass[pair_i]
    mass_j = particles.mass[pair_j]
    larger_idx = np.where(mass_j > mass_i, pair_j, pair_i)
    smaller_idx = np.where(mass_j > mass_i, pair_i, pair_j)

    # perform transfer and mark deletions
    # TODO note that we use temporary arrays for indexing because of KernelParticle bug (GH #2143)
    masses = particles.mass
    states = particles.state

    # transfer mass from smaller to larger and mark smaller for deletion
    masses[larger_idx] += particles.mass[smaller_idx]
    states[smaller_idx] = parcels.StatusCode.Delete

    # TODO use particle variables directly after KernelParticle bug (GH #2143) is fixed
    particles.mass = masses
    particles.state = states

In [ ]:
npart = 800

# Create custom Particle class with extra variable mass
MergeParticle = parcels.Particle.add_variable(
    parcels.Variable("mass", dtype=np.float32)
)

pset = parcels.ParticleSet(
    fieldset=DiffusionFieldSet(),
    pclass=MergeParticle,
    lon=np.random.uniform(-1, 1, size=npart),
    lat=np.random.uniform(-1, 1, size=npart),
    mass=np.random.uniform(0.5, 1.5, size=npart),
)

output_file = parcels.ParticleFile(
    store="MergingParticles.zarr",
    outputdt=np.timedelta64(1, "s"),
)

kernels = [
    parcels.kernels.DiffusionUniformKh,
    Merge,  # Add the Merge kernel defined above
]

pset.execute(
    pyfunc=kernels,
    runtime=np.timedelta64(60, "s"),
    dt=np.timedelta64(1, "s"),
    output_file=output_file,
    verbose_progress=False,
)

In [ ]:
data_xarray = xr.open_zarr("MergingParticles.zarr")

timerange = np.arange(
    np.nanmin(data_xarray["time"].values),
    np.nanmax(data_xarray["time"].values),
    np.timedelta64(1, "s"),
)

fig = plt.figure(figsize=(4, 4), constrained_layout=True)
ax = fig.add_subplot()

ax.set_ylabel("Meridional distance [m]")
ax.set_xlabel("Zonal distance [m]")
ax.set_xlim(-1.1, 1.1)
ax.set_ylim(-1.1, 1.1)

time_id = np.where(data_xarray["time"] == timerange[0])

scatter = ax.scatter(
    data_xarray["lon"].values[time_id],
    data_xarray["lat"].values[time_id],
    s=data_xarray["mass"].values[time_id],
    c="b",
    zorder=1,
)

t = str(timerange[0].astype("timedelta64[s]"))
title = ax.set_title("Particles at t = " + t)


def animate(i):
    t = str(timerange[i].astype("timedelta64[s]"))
    title.set_text("Particles at t = " + t)

    time_id = np.where(data_xarray["time"] == timerange[i])
    scatter.set_offsets(
        np.c_[data_xarray["lon"].values[time_id], data_xarray["lat"].values[time_id]]
    )
    scatter.set_sizes(data_xarray["mass"].values[time_id])

    return (scatter,)


anim = FuncAnimation(fig, animate, frames=len(timerange), interval=100)
plt.close(fig)
anim